In [99]:
import csv
import pandas as pd
import random
from datetime import datetime

f = 'C:/Data/20181229_인천시_교통카드데이터/1-1.거래내역/DI_TBTRDD010_20181101.csv'
num_lines = sum(1 for l in open(f, encoding='UTF-8'))
size = int(num_lines * 0.01)
skip_idx = random.sample(range(1, num_lines), num_lines - size)

In [100]:
df = pd.read_csv(f, skiprows=skip_idx, index_col=None, header=0, encoding='utf-8', low_memory=False)
df.head()

,카드번호,운행출발일시,트랜잭션ID,교통수단코드,환승횟수,노선ID,교통사업자ID,차량ID,사용자구분코드,승차일시,승차정류장ID,하차일시,하차정류장ID,이용객수,통행거리,통행시간
0,dEi5luwGmEWcZdXDQwokhCMBsE8mBWJjfgnL0MFItuM=,~,86,201,0,~,211000000,~,1,20181101181814,222,20181101185500,0208,1,8700,2206
1,AxKIFyHYahavEqwXpnh064mbvY+6mRYvDI7z6GVgvRI=,~,69,201,0,~,211000000,~,1,20181101181853,222,20181101184821,0229,1,8900,1768
2,PpR5XzcDyLvvFZOhiADprj8bqtvKXHXYRuFWYAHN+z8=,~,27,201,0,~,211000000,~,1,20181101185818,222,20181101192347,0230,1,10000,1529
3,ZmReQu4RDePzOK8LCNzos6gGRbCDwRSuv3Lng6wUmHY=,~,19,201,0,~,211000000,~,1,20181101183322,222,20181101184642,0217,1,5500,800
4,O9abhBRsAQSQDusY8y/zBilO1/RwUbvcGsi91xHmPHo=,~,14,201,0,~,211000000,~,1,20181101184016,222,20181101190547,0227,1,6900,1531


In [123]:
with open('IC_ETick.txt', mode='w', newline='') as visum_file:
    visum_writer = csv.writer(visum_file, delimiter=';', quoting=csv.QUOTE_NONE)
    
    visum_writer.writerow(['$VISION'])
    visum_writer.writerow(['* VisumInst'])
    visum_writer.writerow(['* 22.05.12, 18.27.29'])
    visum_writer.writerow(['*'])
    visum_writer.writerow(['*'])
    visum_writer.writerow(['* Tabelle: Versionsblock'])
    visum_writer.writerow(['$VERSION', 'VERSNR', 'FILETYPE', 'LANGUAGE', 'UNIT'])
    visum_writer.writerow(['4.00','Att','Eng','KM'])
    visum_writer.writerow(['*'])
    visum_writer.writerow(['*'])
    visum_writer.writerow(['* Table: PuT path legs'])
    visum_writer.writerow(['$SingleRowSurveyData:No', 'NumPass', 'SurveyLineName', 'InputStopNo', 'InputStopDepTime', 'InputStopDepDay', 'BoardStopNo', 'AlightStopNo', 'DestStopNo', 'OrigStopNo', 'PreTSysCode'])
# 번호	승객수	탑승노선	탑승정류장	탑승시간	탑승일자	하차정류장	환승정류장	최종하차정류장	최초출발정류장	
    visum_writer.writerow(['*'])
    visum_writer.writerow(['*'])
    
    seq_no = 1
    no_dict = {}
    
    for index, row in df.head().iterrows():
        no = no_dict.setdefault(row['카드번호'], seq_no)
        if no == seq_no:
            seq_no = seq_no + 1
        
        dt = str(row['승차일시'])
        InputStopDepTime = datetime.strptime(dt[8:], '%H%M%S').strftime('%H:%M:%S')
        InputStopDepDay = datetime.strptime(dt[:8], '%Y%m%d').strftime('%Y-%m-%d')
        
        visum_writer.writerow([no, row['이용객수'], row['노선ID'], row['승차정류장ID'], 
                              InputStopDepTime, InputStopDepDay, row['승차정류장ID'], row['하차정류장ID'], 
                              row['하차정류장ID'], row['승차정류장ID'], '', ''])